In [0]:
# Databricks Notebook: Silver -> Gold (시간 윈도우 집계)
# 목적: Silver 캔들 데이터를 고정 길이 윈도우로 평균 가격을 집계하여 Gold 테이블로 업서트

from pyspark.sql.functions import col, window, avg, lit

# =========================
# (A) 환경/대상 테이블 설정
# =========================
CATALOG = "demo_catalog"
SCHEMA  = "demo_schema"
SILVER  = f"{CATALOG}.{SCHEMA}.silver_charts"  # 정형 캔들 데이터
GOLD    = f"{CATALOG}.{SCHEMA}.gold_signals"   # 윈도우 집계 결과 저장
WINDOW_SPEC = "5 minutes"                      # 집계 윈도우 크기

# =========================
# (B) Gold 테이블 생성(없으면)
#  - 기존 스키마 유지: window_spec/dt 컬럼 없이 운영
# =========================
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {GOLD} (
  bucket_start  TIMESTAMP,  -- 윈도우 시작 시각
  bucket_end    TIMESTAMP,  -- 윈도우 종료 시각
  symbol        STRING,     -- 자산 심볼
  avg_price     DOUBLE,     -- 윈도우 평균 종가
  latest_signal STRING      -- 후속 신호 컬럼(현재 NULL)
) USING DELTA
""")

# =========================
# (C) 원본 로드 및 전처리
#  - open_time이 유효한 행만 사용
# =========================
silver = spark.table(SILVER).where("open_time IS NOT NULL")

# =========================
# (D) 윈도우 집계 DF 구성
#  - window(open_time, WINDOW_SPEC)로 고정 길이 버킷 생성
#  - 심볼별 평균 종가 계산 → bucket_start/end와 함께 출력
#  - latest_signal은 현재 NULL로 채움(스키마 유지 목적)
#  - 동일 (symbol, bucket_start) 중복 방지
# =========================
agg_df = (
    silver
      .groupBy(window(col("open_time"), WINDOW_SPEC), col("symbol"))
      .agg(avg(col("close")).alias("avg_price"))
      .select(
          col("window.start").alias("bucket_start"),
          col("window.end").alias("bucket_end"),
          col("symbol"),
          col("avg_price")
      )
      .withColumn("latest_signal", col("avg_price")*0 - col("avg_price")*0)  # NULL 생성 트릭
      .dropDuplicates(["symbol", "bucket_start"])
)

# 임시 뷰로 노출해 SQL MERGE에 사용
agg_df.createOrReplaceTempView("gold_upserts")

# =========================
# (E) Delta MERGE (idempotent upsert)
#  - 조인 키: (symbol, bucket_start)
#  - 존재 시 UPDATE, 없으면 INSERT
# =========================
spark.sql(f"""
MERGE INTO {GOLD} AS t
USING gold_upserts AS s
ON  t.symbol       = s.symbol
AND t.bucket_start = s.bucket_start
WHEN MATCHED THEN UPDATE SET
  t.bucket_end     = s.bucket_end,
  t.avg_price      = s.avg_price,
  t.latest_signal  = s.latest_signal
WHEN NOT MATCHED THEN INSERT *
""")

print(f"Gold aggregate complete (window={WINDOW_SPEC})")